<a href="https://colab.research.google.com/github/twinstae/tripReviewAnalysisSystem/blob/master/%ED%81%AC%EB%A1%A4%EB%9F%AC-%EC%A0%84%EC%B2%98%EB%A6%AC/TRAS_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q selenium
!pip install -q bs4
!pip install -q pandas

     |████████████████████████████████| 911kB 2.8MB/s 


In [0]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from pandas import Series, DataFrame
from selenium.webdriver.common.keys import Keys
import time
import re

In [0]:
#키를 파일 이름으로 저장
def data_to_csv(dict_n):
    '''
    입력 : 딕셔너리 데이터
    출력 : csv파일 형태로 원시 자료 폴더에 출력.
    '''
    git_address = 'C:/Users/taehee/Documents/GitHub'
    for key, an_attracrion_dict in dict_n.items():
        an_attracrion_dict.to_csv(git_address + '/tripReviewAnalysisSystem/크롤러-전처리/원시자료/' + key + '.csv', mode = 'w')

In [0]:
def reviews(sample_size):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 표본 리뷰의 숫자자.
    맥락 : 리뷰를 추출할 즐길거리 페이지에 있다.
    출력 : 한 즐길거리에 대한 리뷰를 모아놓은 데이터 프레임

    각 리뷰에서 새로운 항목을 추출하고 싶다면, columns와 for문을 모두 수정해야한다.
    '''
    reviews = []
    columns = ['star_point','title','text','Date of experience']
    #,'reviewer_address'
        
        
    #리뷰 크기를 텍스트로 찾아와, 콤마를 제거하여, 정수형으로 저장.
    review_size_text = driver.find_element_by_class_name('attractions-community-content-TabBarContent__tabCount--2hTdj').text
    review_size = int(re.sub(',','', str(review_size_text)))

    n = 5
    count = 0
    while n < sample_size:
        try:
            foot = driver.find_element_by_class_name('location-review-review-list-parts-ReviewScrollController__container--3XtvE')
            show_more = foot.find_element_by_tag_name('button')  
            driver.execute_script("arguments[0].click();", show_more)
            n += 5
        except:
            if count > 3:
                break
            count +=1
            time.sleep(2)
                
    review_cards = driver.find_elements_by_class_name('location-review-card-Card__ui_card--2Mri0')

    for review_card in review_cards:
        try:
            read_more = review_card.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
            read_more.click()
        except:
            print('error')
            continue
    
    # 드라이버 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_elements = soup.find_all(class_ = 'location-review-card-Card__ui_card--2Mri0 location-review-card-Card__card--o3LVm location-review-card-Card__section--NiAcw')
    
    for review_element in review_elements:
        try:
            review = [
                int(review_element.find('span',class_='ui_bubble_rating')['class'][1][-2:]) / 10,
                review_element.find('a', class_='location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT').find('span').find('span').text,
                review_element.find('q', class_='location-review-review-list-parts-ExpandableReview__reviewText--gOmRC').find('span').text,
                review_element.find('span', class_='location-review-review-list-parts-EventDate__event_date--1epHa').contents[1]
                #,review_element.find('span', class_='social-member-common-MemberHometown__hometown--3kM9S').contents[1]
            ]
        except IndexError:
            #아무도 helpful이나 vote를 누르지 않은 에러, 무시하고 계속
            continue
        except AttributeError:
            #주소를 공개하지 않는 에러, 무시하고 계속
            continue
        reviews.append(review)

    return DataFrame(reviews, columns = columns)

In [7]:
def find_address():
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  contact = soup.find_all(class_ = 'attractions-contact-card-ContactCard__contactRow--3Ih6v')
  address_block = contact[0]
  address_spans = address_block.contents
  address_span = address_spans[1]
  address = address_span.text
  return address

"""data_test = [names, location]

df_list = pd.DataFrame(data_test, columns = columns)
df_list.to_csv('crawler test.csv')
"""

"data_test = [names, location]\n\ndf_list = pd.DataFrame(data_test, columns = columns)\ndf_list.to_csv('crawler test.csv')\n"

In [0]:
def load_attraction(link, fuction):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 어떤 즐길거리 번호 int, 이름 string 예: "Gyeongbokgung Palace"
    출력 : 어트랙션의 리뷰 모음. data.frame
    '''
    #새 창을 열고 링크로 간다.
    # 슬립...매우매우중요이거없으면무조건에러남.이거하나로3시간머리싸맸음.
    
    time.sleep(5)
    original_window = driver.current_window_handle
    driver.execute_script("arguments[0].click();", link)
    
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break
    
    #즐길거리 별로 샘플사이즈를 어떻게 정할 것인지?    
    result = function()
    
    if result['text'].isnull().sum() != 0:
        print(result['text'].isnull().sum())
        result.head()

    #창을 닫고 리스트 창으로 돌아간다
    driver.close();
    driver.switch_to.window(original_window)
    driver.current_window_handle;
    
    return result
    #result.head()

In [0]:
def load_page(start, end, function):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 크롤링할 페이지 내 시작 번호와 끝 번호
    출력 : 페이지의 리뷰들을 csv파일로 출력
    '''

    #모든 여행지의 리스트를 만든다
    is_first = True
    print("I am in the load_page")
    if driver.current_url == 'https://www.tripadvisor.com/Attractions-g294197-Activities-Seoul.html':
        main_page = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__container--3eHZU')
        attraction_elements = main_page.find_elements_by_class_name("attractions-attraction-overview-pois-PoiInfo__info--239IR")
        is_first = True
    else:
        main_page = driver.find_element_by_class_name('scrollAdMain')
        attraction_elements = main_page.find_elements_by_class_name('listing_info')
        is_first = False
    print("I found the elements!")

    attraction_names = []
    for attraction_element in attraction_elements[start:end]:      
        if is_first:
            link = attraction_element.find_element_by_tag_name('h3')
            attraction_name = link.text[link.text.find('.')+2 :]
        else:
            div_class = attraction_element.find_element_by_class_name('tracking_attraction_title')
            link = div_class.find_element_by_tag_name('a')
            attraction_name = link.text
        attraction_names.append((attraction_name, link))   
    print("I found the attraction_names")
        
    attraction_dict = {}
    for attraction_name, link in attraction_names:
        if len(attraction_dict) < end:
            stime = time.time()
            attraction_dict[attraction_name] = load_attraction(link, function)
            print(attraction_name,  " " + str(time.time() -stime) + "초 걸림 ", len(attraction_dict[attraction_name]))
            if len(attraction_dict) % 5 == 0:
                data_to_csv(attraction_dict)
                print("저장했어요!")
        else:
            data_to_csv(attraction_dict)
            print("저장했어요!")
            break

In [0]:
def is_first_page(driver):
    try:
        time.sleep(1)
        show = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__see_more--2Vsb-')
        driver.execute_script("arguments[0].click();", show)
        is_first_page = True
        print("it is first page!")
    except NoSuchElementException:
        print('continue!')
        is_first_page = False
    except:
        print("exception")

In [0]:
def get_start_page(is_first_page, start, end):
    if is_first_page:
        selected_page = 1
        page_numbers = driver.find_element_by_class_name('attractions-attraction-overview-main-Pagination__container--PUXGq')
        if start // 30 > 0:
            if start // 30 > 6:
                target_num = 6
            else:
                target_num = (start // 30) + 1
            target_page = page_numbers.find_element_by_link_text(str(target_num))
            time.sleep(5)
            driver.execute_script("arguments[0].click();", target_page)
            time.sleep(3)
            selected_page = target_num
            page_numbers = driver.find_element_by_class_name('pageNumbers')
            is_first_page = False
        else:
            selected_page = int(soup.find("div", class_='pageNumbers').find(class_ = 'current').text)
            page_numbers = driver.find_element_by_class_name('pageNumbers')


In [0]:
def crawl_pages(is_first_page, start, end, function):
    keep_going = True    
    while keep_going:
        # 현재 페이지 번호 알아내기

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        print("I made a soup")
        selected_page, page_numbers = get_start_page(is_firt_page, start, end)
            
                # 시작점이 뒷 페이지면 지나간다
        if start <= selected_page * 30:
            strat_in_page = start-(selected_page-1)*30
            if strat_in_page < 0:
                strat_in_page = 0
            assert strat_in_page <= 30
            
            end_in_page = end-(selected_page-1)*30
            if end_in_page > 30:
                end_in_page = 30                
            assert 0 <= end_in_page
            if is_first_page:
                is_first_page = False
                
            load_page(strat_in_page, end_in_page, function)
        
        # 끝나는 지점을 지나갔으면 루프를 끝낸다.
        if (selected_page - 1) * 30 >= end:
            keep_going = False
            break
            
        print("I find the current page")

        # 다음 페이지로 넘어가기
        time.sleep(5)
        next_num = str(selected_page + 1)
        next_page = page_numbers.find_element_by_link_text(next_num)
        driver.execute_script("arguments[0].click();", next_page)

In [0]:
def crawl_addresses(start, end):
    '''
    입력 : 크롤링할 즐길거리의 시작 번호와 끝 번호
    출력 : 해당 구간의 즐길거리 리뷰들csv로 출력
    '''    
    #처음 한 번만 see more을 누른다.
    is_first_page = is_first_page(driver)
    
    time.sleep(5)
    crawl_pages(is_first_page, start, end)

In [14]:
# 언어를 영어 캐나다로 설정. 이래야 show more버튼이 나오는 줄 알았는데...
options = webdriver.ChromeOptions()
options.add_argument("lang=en-CA")
#이미지를 보지 않음
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver_address = 'C:/Users/taehee/Downloads/chromedriver_win32/chromedriver'

is_show_more = False
while not is_show_more:
    driver = webdriver.Chrome(driver_address, chrome_options=options)
    wait = WebDriverWait(driver, timeout=2)
    driver.implicitly_wait(5)
    # tripadvisor 경복궁에 접근한다.
    driver.get('https://www.tripadvisor.com/Attraction_Review-g294197-d324888-Reviews-Gyeongbokgung_Palace-Seoul.html')
    time.sleep(1)
    try:
        foot = driver.find_element_by_class_name('location-review-review-list-parts-ReviewScrollController__container--3XtvE')
        show_more = foot.find_element_by_tag_name('button')
    except:
        driver.close();
        time.sleep(1)
        continue
    is_show_more =True

#원래 있던 리스트 창을 저장해둔다
time.sleep(10)
driver.get('https://www.tripadvisor.com/Attractions-g294197-Activities-Seoul.html')


crawl_function(function = find_addresss)

#dict_n() => load_page() => load_reviews() => reviews()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


WebDriverException: ignored